# Modules

### Module basics

In [1]:
x = 1

1

In [2]:
module Ant
    x = "not 1"
end

Main.Ant

In [3]:
x

1

In [4]:
Ant.x

"not 1"

In [5]:
module Bee
    x = ["also", "not", "one"]
end

Main.Bee

In [6]:
Bee.x

3-element Array{String,1}:
 "also"
 "not" 
 "one" 

In [7]:
Ant.x

"not 1"

In [8]:
x

1

In [10]:
x = "some string"

"some string"

In [11]:
Ant.x = "dead ant"

ErrorException: cannot assign variables in other modules

**Tricky hack:** evaling into modules

In [12]:
@eval Ant x = "dead ant"

"dead ant"

In [14]:
Ant.eval(:(x = "ex ant"))

"ex ant"

In [15]:
Ant.x

"ex ant"

**Modules all the way down:** You're always in a module. By default it's just `Main`:

In [16]:
@__MODULE__

Main

In [17]:
@eval Ant @__MODULE__

Main.Ant

In [18]:
module Boar
    @show @__MODULE__
end

#= In[18]:2 =# @__MODULE__() = Main.Boar


Main.Boar

In [20]:
@show x^3

x ^ 3 = "some stringsome stringsome string"


"some stringsome stringsome string"

## Importing modules and bindings

Problem statement:
```jl
module Cat
    lives = 9
end

module Dog
    # how can Dog see Cat?
    # how can Dog see Cat.lives?
end
```
The `import` keyword to the rescue!

In [21]:
module Cat
    lives = 9
end

Main.Cat

In [22]:
Cat.lives

9

In [23]:
import .Cat: lives

In [28]:
module Dog
    import .Cat
end

Main.Dog

In [29]:
module Dog
    import Main.Cat
end

Main.Dog

In [27]:
Dog.Cat

Main.Cat

In [30]:
Dog.Cat.lives

9

In [31]:
module Elk
    import ..Cat: lives
end

Main.Elk

In [32]:
Elk.lives

9

In [33]:
Elk.Cat

UndefVarError: UndefVarError: Cat not defined

In [34]:
import .Cat: lives

In [35]:
lives

9

In [36]:
@eval Cat lives = 11

11

In [37]:
lives

11

In [38]:
Elk.lives

11

In [40]:
lifes = Cat.lives

11

In [41]:
@eval Cat lives = 12

12

In [42]:
lives

12

In [43]:
lifes

11

### But I want to import a lot of things

The `export` and `using` to the rescue!

In [46]:
module Fox
    export ears, tail, feet, eyes, nose, wits

    ears = "pointy"
    tail = "fluffy"
    feet = 4
    eyes = "sharp"
    nose = "keen"
    wits = "keener"
    private = 0
end

Main.Fox

In [45]:
ears

UndefVarError: UndefVarError: ears not defined

In [47]:
using .Fox

In [48]:
ears

"pointy"

In [49]:
nose

"keen"

In [50]:
wits

"keener"

In [51]:
private

UndefVarError: UndefVarError: private not defined

In [52]:
@eval Fox feet = 5 # too much radiation?

5

In [53]:
feet

5

In [54]:
module Gnu
    using ..Fox

    tail = "scraggly"
    nose = "flared"
    wits = "dull"
end

Main.Gnu

In [55]:
Fox.tail

"fluffy"

In [56]:
Gnu.tail

"scraggly"

In [57]:
Gnu.feet

5

In [58]:
Gnu.ears

"pointy"

In [59]:
tail = "a function that gives the rest of the things"

"a function that gives the rest of the things"

In [60]:
feet = "an imperial measure"

ErrorException: cannot assign variable Fox.feet from module Main

### using: name conflicts

In [69]:
module Bird
    export wing, beak
    wing = "feathered"
    beak = "hooked"
end

module Airplane
    export wing, cockpit
    wing = "metallic"
    cockpit = "front"
end

Main.Airplane

In [70]:
using .Bird
using .Airplane
using .Bird, .Airplane

In [64]:
wing

UndefVarError: UndefVarError: wing not defined

In [65]:
Bird.wing

"feathered"

In [66]:
Airplane.wing

"metallic"

In [67]:
import .Bird: wing

In [68]:
wing

"feathered"

In [71]:
beak

"hooked"

In [72]:
cockpit

"front"

## Submodules

In [73]:
module Hen
    cluck() = "Puk Puk Pukaaak"

    module Egg
        yolk = 12.5
    end
    module Brood
        import ..Egg
    end
end

Main.Hen

In [74]:
Hen.Egg.yolk

12.5

In [75]:
Hen.Brood.Egg

Main.Hen.Egg

In [76]:
Hen.cluck()

"Puk Puk Pukaaak"

## Including other files

Suppose the parts of the `Hen` modules were much bigger and it got unweildy to have them all in a single file. We can split it up into multiple files and then use `include` to include source files from inside the module.

In [77]:
;cat cluck.jl

println("Loading ", @__FILE__)

cluck() = "Puk Puk Pukaaak"

export cluck


In [78]:
;cat Egg.jl

println("Loading ", @__FILE__)

module Egg
    yolk = 12.5
end


In [79]:
;cat Brood.jl

println("Loading ", @__FILE__)

module Brood
    import ..Egg
end


In [82]:
module Hen
    include("cluck.jl")
    include("Egg.jl")
    include("Brood.jl")
end

Loading /Users/stefan/training/ORNLTraining/Day3/Modules/cluck.jl
Loading /Users/stefan/training/ORNLTraining/Day3/Modules/Egg.jl
Loading /Users/stefan/training/ORNLTraining/Day3/Modules/Brood.jl


Main.Hen

In [83]:
Hen.Egg.yolk

12.5

In [84]:
Hen.cluck()

"Puk Puk Pukaaak"

In [85]:
using .Hen

In [86]:
cluck()

"Puk Puk Pukaaak"

**Note:** if you're reloading and editing code a lot from the REPL, please check out https://timholy.github.io/Revise.jl/latest/ — it totally transforms the way you can do this kind of workflow!

## Extending functions from other modules

In [87]:
cluck() = "cluck, cluck, cluck, CLUCKAAAAAWWWWK!"

ErrorException: error in method definition: function Hen.cluck must be explicitly imported to be extended

In [88]:
Hen.cluck() = "cluck, cluck, cluck, CLUCKAAAAAWWWWK!"

In [89]:
Hen.cluck()

"cluck, cluck, cluck, CLUCKAAAAAWWWWK!"

In [90]:
cluck()

"cluck, cluck, cluck, CLUCKAAAAAWWWWK!"

In [92]:
import .Hen: cluck

In [93]:
cluck() = "bkack"

cluck (generic function with 1 method)

In [94]:
Hen.cluck()

"bkack"

## Review of function and keywords

- `include`: like `source` in the shell or `#include` in C, just read the expressions in a file and evaluate them in the current module. This is the lowest level way of including files.

- `import`: get names from other modules—the modules have to already exist.

- `using`: like import but "auto imports" any names that the module exports.

- `export`: use this inside a module to export names.

# Packages versus modules

When you do this, you're importing a pre-existing module:
```jl
import .Moo
```
On the other hand, when you do this without the dot, you're loading a package:
```jl
import Moo
```